In [1]:
# 安装工具链
%pip install uv
!uv tool install "huggingface_hub[cli]"


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 61.6 MB/s eta 0:00:00
Resolved 17 packages in 86ms
Prepared 17 packages in 152ms
Installed 17 packages in 16ms
 + certifi==2025.8.3
 + charset-normalizer==3.4.3
 + filelock==3.19.1
 + fsspec==2025.9.0
 + hf-xet==1.1.10
 + huggingface-hub==0.35.1
 + idna==3.10
 + inquirerpy==0.3.4
 + packaging==25.0
 + pfzy==0.3.4
 + prompt-toolkit==3.0.52
 + pyyaml==6.0.3
 + requests==2.32.5
 + tqdm==4.67.1
 + typing-extensions==4.15.0
 + urllib3==2.5.0
 + wcwidth==0.2.14
Installed 3 executables: hf, huggingface-cli, tiny-agents


In [2]:
# 获取代码
!wget https://github.com/index-tts/index-tts/archive/refs/heads/main.zip
!unzip main.zip
!mv index-tts-main index-tts
!rm main.zip


--2025-09-26 03:14:40--  https://github.com/index-tts/index-tts/archive/refs/heads/main.zip
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/index-tts/index-tts/zip/refs/heads/main [following]
--2025-09-26 03:14:40--  https://codeload.github.com/index-tts/index-tts/zip/refs/heads/main
Resolving codeload.github.com (codeload.github.com)... 20.205.243.165
Connecting to codeload.github.com (codeload.github.com)|20.205.243.165|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘main.zip’

main.zip                [           <=>      ]  31.73M  11.9MB/s    in 2.7s    

2025-09-26 03:14:43 (11.9 MB/s) - ‘main.zip’ saved [33269072]

Archive:  main.zip
84a5ef97b80a0cff368bfe921e20e826542df2a8
   creating: index-tts-main/
  inflating: index-tts-main/.gitattributes  
   cr

In [3]:
# 安装依赖
!cd /content/index-tts/ && uv sync --all-extras

Using CPython 3.10.12 interpreter at: /usr/bin/python3.10
Creating virtual environment at: .venv
Resolved 190 packages in 1ms
Prepared 178 packages in 1m 33s
Installed 178 packages in 1.71s
Prepared 1 package without build isolation in 33.58s
Installed 1 package in 21ms
 + absl-py==2.3.1
 + accelerate==1.8.1
 + aiofiles==24.1.0
 + annotated-types==0.7.0
 + antlr4-python3-runtime==4.9.3
 + anyio==4.10.0
 + argbind==0.3.9
 + asttokens==3.0.0
 + audioread==3.0.1
 + brotli==1.1.0
 + cachetools==5.5.2
 + certifi==2025.8.3
 + cffi==2.0.0
 + charset-normalizer==3.4.3
 + click==8.2.1
 + cn2an==0.5.22
 + contourpy==1.3.2
 + cycler==0.12.1
 + cython==3.0.7
 + decorator==5.2.1
 + deepspeed==0.17.1
 + descript-audiotools==0.7.2
 + distance==0.1.3
 + docstring-parser==0.17.0
 + einops==0.8.1
 + exceptiongroup==1.3.0
 + executing==2.2.1
 + fastapi==0.116.2
 + ffmpeg-python==0.2.0
 + ffmpy==0.6.1
 + filelock==3.19.1
 + fire==0.7.1
 + flatten-dict==0.4.2
 + fonttools==4.60.0
 + fsspec==2025.9.0
 + fut

In [4]:
# 下载模型 checkpoints
!hf download IndexTeam/IndexTTS-2 --local-dir=/content/index-tts/checkpoints

Fetching 20 files:   0% 0/20 [00:00<?, ?it/s]Downloading 'feat2.pt' to '/content/index-tts/checkpoints/.cache/huggingface/download/jhr_Oe-8eUDBi3ePfijGwmdsvlE=.9c4292e96dee535aea9a6206e9a0c856dd578dde9212acdb16dd3ada4d12bf80.incomplete'

.gitattributes: 0.00B [00:00, ?B/s]Downloading 'config.yaml' to '/content/index-tts/checkpoints/.cache/huggingface/download/_ouC4VcfA9-o6nAiMaExMJW7X2A=.98d16c3892bd2321e7da71a379ecc8e1fa4dd84e.incomplete'


.gitattributes: 1.73kB [00:00, 1.17MB/s]Downloading 'qwen0.6bemo4-merge/Modelfile' to '/content/index-tts/checkpoints/.cache/huggingface/download/qwen0.6bemo4-merge/-T10lO8x1ud2PVxfECgsRvh55vU=.b1bed86c7c360343f16d4580b7fb6a19182da273.incomplete'

Download complete. Moving file to /content/index-tts/checkpoints/.gitattributes
README.md: 2.50kB [00:00, 1.61MB/s]
Download complete. Moving file to /content/index-tts/checkpoints/README.md
Fetching 20 files:   5% 1/20 [00:00<00:06,  2.99it/s]
config.yaml: 2.88kB [00:00, 5.54MB/s]

Modelfile: 100% 360/36

In [5]:
# 为了直接利用 Gradio 的内网穿透功能
# 需要对 index—tts/webui.py 进行修改

# 如果因为 IndexTTS 官方 GitHub 仓库的更改造成本代码不可用
# 还可以手动修改 webui.py ，在 launch 函数调用中加上 `share = True`


%cd /content/index-tts

launch_line = 'demo.launch(server_name=cmd_args.host, server_port=cmd_args.port)'
launch_line_with_share = 'demo.launch(server_name=cmd_args.host, server_port=cmd_args.port, share=True)'

with open("webui.py", encoding='utf8') as f:
  webui_with_share = []
  for line in f.readlines():
    if line.strip() == launch_line:
      webui_with_share.append(line.replace(launch_line, launch_line_with_share))
    else:
      webui_with_share.append(line)

  with open("webui_with_share.py", 'w', encoding='utf8') as f:
    f.writelines(webui_with_share)

/content/index-tts


In [ ]:
# 启动

%cd /content/index-tts/
!uv run webui_with_share.py

/content/index-tts
>> GPT weights restored from: ./checkpoints/gpt.pth
GPT2InferenceModel has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
preprocessor_config.json: 100% 275/275 [00:00<00:00, 2.31MB/s]
config.json: 1.87kB [00:00,